In [1]:
import pandas as pd
import numpy as np
import warnings
import time
from sklearn.model_selection import StratifiedKFold, ParameterGrid, cross_val_score, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Tree-based Models (Scikit-learn: CPU Multi-core)
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    RandomForestClassifier,
    AdaBoostClassifier,
    GradientBoostingClassifier,
    BaggingClassifier,
    ExtraTreesClassifier
)
# 외부 라이브러리 모델
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# CPU 모델
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.svm import NuSVC as SklearnNuSVC, SVC as SklearnSVC, LinearSVC as SklearnLinearSVC
from sklearn.neighbors import KNeighborsClassifier as SklearnKNN

# cuML (GPU) 로드 시도
try:
    import cuml
    from cuml.svm import SVC as cuSVC
    from cuml.svm import LinearSVC as cuLinearSVC
    from cuml.neighbors import KNeighborsClassifier as cuKNN
    print(f"✅ [System] cuML(GPU) 로드 성공 (Version: {cuml.__version__})")
    GPU_AVAILABLE = True
except:
    print("⚠️ [System] cuML을 찾을 수 없습니다. GPU 모델은 CPU 모델로 대체됩니다.")
    GPU_AVAILABLE = False

warnings.filterwarnings('ignore')

# =========================================================
# 1. 데이터 로드 및 전처리 (Pandas DataFrame 유지)
# =========================================================
file_path = "Dry_Eye_Dataset_preprocessed.csv"
target_col = 'Dry Eye Disease'
print("\n>> [Data] 데이터 로딩 중...")

try:
    df = pd.read_csv(file_path)
    if df.isnull().sum().sum() > 0: df.fillna(0, inplace=True)
except FileNotFoundError:
    print("[Error] 파일을 찾을 수 없어, 테스트용 랜덤 데이터를 생성합니다.")
    df = pd.DataFrame(np.random.randn(20000, 25), columns=[f'feature_{i}' for i in range(25)])
    df[target_col] = np.random.randint(0, 2, 20000)

X_raw = df.drop(columns=[target_col])
y_raw = df[target_col]

# SHAP 대신 Correlation Feature Selection을 위해 전체 X, y와 train/test set 모두 준비
X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(X_raw, y_raw, test_size=0.2, random_state=42)

# 튜닝에 사용할 데이터 (Pandas DataFrame 형태 유지)
X_for_tuning = X_raw
y_for_tuning = y_raw

# 타겟 인코딩 (필요한 경우)
le = LabelEncoder()
y_encoded_all = le.fit_transform(y_raw)
y_int_for_tuning = pd.Series(y_encoded_all, index=y_raw.index, name=target_col) # Sklearn CV에 사용할 인코딩된 y

print(f"   - 데이터 준비 완료: {X_raw.shape}")

# =========================================================
# 2. 상관관계 기반 특성 선택 및 정렬
# =========================================================
print("\nCalculating Feature Importance using Correlation...")
df_combined = pd.concat([X_raw, y_raw], axis=1)

# 타겟 변수와의 상관관계 계산
correlation_with_target = df_combined.corr()[target_col].drop(target_col)

# 절대값 기준 정렬
abs_correlation_sorted = correlation_with_target.abs().sort_values(ascending=False)
sorted_features = abs_correlation_sorted.index.tolist()

print("\n[Feature Importance Ranking (Correlation)]")
print(f"{'Rank':<5} | {'Feature Name':<30} | {'Correlation':<15}")
print("-" * 60)
for idx, (name, score) in enumerate(abs_correlation_sorted.items()):
    print(f"{idx + 1:<5} | {name:<30} | {score:.5f}")

print("\n" + "="*85 + "\n")

# =========================================================
# 3. 통합 튜닝 함수 (Pandas DataFrame 사용하도록 수정)
# =========================================================

def run_tuning_and_feat_select(model_name, model_class, param_grid, device, X_df, y_series_int, sorted_features, cv):
    print(f"\n" + "="*80)
    print(f" 🚀 Running: {model_name} ({device}) - Feat Select & Grid Search")
    print(f"="*80)

    start_time = time.time()
    overall_best_score = -1
    overall_best_params = None
    overall_best_k = -1

    max_k = min(10, len(sorted_features))
    param_list = list(ParameterGrid(param_grid))
    total_runs = max_k * len(param_list)
    run_count = 0

    # Sklearn/CPU 모델은 인코딩된 y_int를, cuML/GPU 모델은 Series 형태의 y_int를 사용
    y_data = y_series_int

    for k in range(1, max_k + 1):
        current_features = sorted_features[:k]

        # ⭐️ Pandas Column Selection 사용
        X_sel = X_df[current_features]

        for params in param_list:
            run_count += 1
            fold_scores = []

            try:
                # Cross Validation Loop
                for fold_idx, (train_idx, val_idx) in enumerate(cv.split(X_sel, y_data)):
                    # ⭐️ Pandas DataFrame/Series 슬라이싱 사용
                    X_train = X_sel.iloc[train_idx]
                    X_val = X_sel.iloc[val_idx]
                    y_train = y_data.iloc[train_idx]
                    y_val = y_data.iloc[val_idx]

                    # cuML 모델을 위해 X_train, y_train을 NumPy/float32로 변환 (필요한 경우)
                    if device == "GPU":
                        X_train = np.ascontiguousarray(X_train.values, dtype=np.float32)
                        X_val = np.ascontiguousarray(X_val.values, dtype=np.float32)
                        y_train = np.ascontiguousarray(y_train.values, dtype=np.float32)
                        y_val_np = np.ascontiguousarray(y_val.values, dtype=np.float32)
                    else:
                        # Scikit-learn CPU 모델은 DataFrame과 Series를 잘 처리함
                        y_val_np = y_val.values

                    current_clf = model_class(**params)
                    current_clf.fit(X_train, y_train)

                    preds = current_clf.predict(X_val)

                    # cuML/XGBoost-GPU 출력을 NumPy/CPU로 변환
                    if hasattr(preds, 'to_numpy'): preds = preds.to_numpy()
                    elif hasattr(preds, 'values'): preds = preds.values.get()
                    elif hasattr(preds, 'get'): preds = preds.get()

                    # 평가 (y_val_np는 int/float 둘 중 하나)
                    fold_scores.append(accuracy_score(y_val_np.astype(np.int64), preds.astype(np.int64)))

                avg_score = np.mean(fold_scores)

                print(f"   [{run_count}/{total_runs}] Feats: {k:<2} | Params: {params}  👉 Acc: {avg_score:.4f}")

                if avg_score > overall_best_score:
                    overall_best_score = avg_score
                    overall_best_params = params
                    overall_best_k = k

            except Exception as e:
                # print(f"   [{run_count}/{total_runs}] Feats: {k:<2} | Params: {params}  👉 Failed ({str(e)[:50]})")
                continue

    elapsed = time.time() - start_time

    print("-" * 80)
    if overall_best_score != -1:
        print(f" ✅ {model_name} Finished.")
        print(f"    - Best Accuracy: {overall_best_score:.4f}")
        print(f"    - Best Features (K): {overall_best_k} ({', '.join(sorted_features[:overall_best_k])})")
        print(f"    - Best Params: {overall_best_params}")
    else:
        print(f" ❌ {model_name} Failed (All Error).")
    print("-" * 80)

    return {
        "Model": model_name,
        "Device": device,
        "Best Accuracy": overall_best_score,
        "Best Features (K)": overall_best_k,
        "Best Parameters": overall_best_params,
        "Time (s)": round(elapsed, 2)
    }


# =========================================================
# 4. 모델 설정 및 실행
# =========================================================
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
results = []
DEFAULT_PARAMS = {'random_state': [42]}

# --- 모델 및 파라미터 설정 (그리드 확장 및 안정화된 버전 유지) ---
model_configs = [
    ("Decision Tree", DecisionTreeClassifier, {'max_depth': [3, 5, 10], 'min_samples_split': [2, 5], **DEFAULT_PARAMS}, "CPU"),
    ("Random Forest", RandomForestClassifier, {'n_estimators': [50, 100], 'max_depth': [5, 10], 'n_jobs': [-1], **DEFAULT_PARAMS}, "CPU"),
    ("AdaBoost", AdaBoostClassifier, {'n_estimators': [50, 100], 'learning_rate': [0.1, 1.0], **DEFAULT_PARAMS}, "CPU"),

    # Gradient Boosting 그리드 확장
    ("Gradient Boosting", GradientBoostingClassifier, {'n_estimators': [50, 100, 150], 'max_depth': [3, 5], 'learning_rate': [0.1, 0.05], **DEFAULT_PARAMS}, "CPU"),

    ("Bagging", BaggingClassifier, {'n_estimators': [10, 20], 'n_jobs': [-1], **DEFAULT_PARAMS}, "CPU"),
    ("Extra Trees", ExtraTreesClassifier, {'n_estimators': [50, 100], 'max_depth': [5, 10], 'n_jobs': [-1], **DEFAULT_PARAMS}, "CPU"),

    # XGBoost (GPU) 그리드 확장
    ("XGBoost", XGBClassifier, {
        'n_estimators': [50, 100, 150],
        'learning_rate': [0.1, 0.05],
        'tree_method': ['hist'], 'device': ['cuda'],
        'eval_metric': ['logloss'], 'use_label_encoder': [False],
        'verbosity': [0], 'n_jobs': [-1], **DEFAULT_PARAMS
    }, "GPU"),

    # LightGBM (CPU) 그리드 확장
    ("LightGBM", LGBMClassifier, {
        'n_estimators': [50, 100, 150],
        'learning_rate': [0.1, 0.05],
        'verbose': [-1], 'n_jobs': [-1],
        **DEFAULT_PARAMS
    }, "CPU"),
]

# Reference 모델 추가
if GPU_AVAILABLE:
    model_configs.extend([
        ("SVC", cuSVC, {'C': [1, 10], 'kernel': ['rbf'], 'gamma': ['scale']}, "GPU"),
        ("LinearSVC", cuLinearSVC, {'C': [1], 'loss': ['squared_hinge']}, "GPU"),
        ("KNN", cuKNN, {'n_neighbors': [5, 9]}, "GPU"),
    ])
else:
    model_configs.extend([
        ("SVC", SklearnSVC, {'C': [1, 10], 'kernel': ['rbf'], 'gamma': ['scale'], 'n_jobs': [-1], **DEFAULT_PARAMS}, "CPU"),
        ("LinearSVC", SklearnLinearSVC, {'C': [1], 'loss': ['squared_hinge'], **DEFAULT_PARAMS}, "CPU"),
        ("KNN", SklearnKNN, {'n_neighbors': [5, 9]}, "CPU"),
    ])

# CPU Only Models
model_configs.extend([
    ("NuSVC", SklearnNuSVC, {'nu': [0.1, 0.5], 'kernel': ['rbf'], **DEFAULT_PARAMS}, "CPU"),
    ("LDA", LinearDiscriminantAnalysis, {'solver': ['svd', 'lsqr']}, "CPU"),
    ("QDA", QuadraticDiscriminantAnalysis, {'reg_param': [0.0, 0.5]}, "CPU"),
])

# 실행 루프 (Pandas DataFrame 사용)
for model_name, model_class, param_grid, device_type in model_configs:
    results.append(run_tuning_and_feat_select(
        model_name,
        model_class,
        param_grid,
        device_type,
        X_for_tuning, # Pandas DataFrame
        y_int_for_tuning, # Pandas Series (Encoded)
        sorted_features,
        skf
    ))

# =========================================================
# 5. 최종 종합 결과 출력
# =========================================================
if results:
    res_df = pd.DataFrame(results)

    res_df_filtered = res_df[res_df['Best Accuracy'] != -1]
    res_df_sorted = res_df_filtered.sort_values(by='Best Accuracy', ascending=False).reset_index(drop=True)

    print("\n\n")
    print("#"*100)
    print(" 🏆 FINAL LEADERBOARD (Correlation Feature Selection + Hyperparameter Tuning) 🏆")
    print("#"*100)

    print(res_df_sorted[['Model', 'Device', 'Best Accuracy', 'Best Features (K)', 'Best Parameters']])

    if not res_df_sorted.empty:
        winner = res_df_sorted.iloc[0]
        print("\n" + "="*100)
        print(f" 🥇 Absolute Best Model: {winner['Model']} ({winner['Device']})")
        print(f"    Accuracy: {winner['Best Accuracy']:.6f}")
        print(f"    Features (K): {winner['Best Features (K)']}")
        print(f"    Parameters: {winner['Best Parameters']}")
        print("="*100)
    else:
        print("\n⚠️ 모든 모델에서 에러가 발생하여 유효한 결과가 없습니다.")

✅ [System] cuML(GPU) 로드 성공 (Version: 25.10.00)

>> [Data] 데이터 로딩 중...
   - 데이터 준비 완료: (20000, 26)

Calculating Feature Importance using Correlation...

[Feature Importance Ranking (Correlation)]
Rank  | Feature Name                   | Correlation    
------------------------------------------------------------
1     | Discomfort Eye-strain          | 0.10987
2     | Redness in eye                 | 0.10341
3     | Itchiness/Irritation in eye    | 0.10130
4     | Average screen time            | 0.02249
5     | Gender                         | 0.01632
6     | Smart device before bed        | 0.01332
7     | Alcohol consumption            | 0.01126
8     | Systolic_BP                    | 0.00729
9     | Height                         | 0.00644
10    | Daily steps                    | 0.00566
11    | Stress level                   | 0.00548
12    | Age                            | 0.00526
13    | Smoking                        | 0.00513
14    | Diastolic_BP                   | 0.00505
1